# Import, preprocess, and store data split

## Import relevant packages

In [55]:
import torch
from torch_geometric.datasets import Coauthor, Planetoid, WikipediaNetwork
from torch_geometric.transforms import RandomLinkSplit

import pickle
import bz2
import time

torch.manual_seed(10)

## Import homogeneous datasets

In [6]:
root = '../data'
wiki_datasets = ["chameleon","crocodile"]
planetoid_dataset = "PubMed"
coauthor_dataset = "CS"

wiki_chameleon = WikipediaNetwork(root=root, name=wiki_datasets[0]).data
wiki_crocodile = WikipediaNetwork(root=root, name=wiki_datasets[1], geom_gcn_preprocess=False).data
pubmed = Planetoid(root=root, name=planetoid_dataset).data
cs = Coauthor(root=root, name=coauthor_dataset).data


Processing...
Done!
Processing...
Done!
Processing...
Done!


## Split each dataset and save

In [56]:
rlp = RandomLinkSplit()
def split_save(data):
    train_data, valid_data, test_data = rlp(data)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    names =['train_data_'+timestr,'valid_data_'+timestr,'test_data_'+timestr]
    data = [train_data, valid_data, test_data]
    for data, name in zip(data,names):
        pickle.dump(data, bz2.BZ2File('../data/{0}.p'.format(name),'wb'))

data = [wiki_chameleon, wiki_crocodile, pubmed, cs]
for set in data:
    split_save(set)

In [57]:
import os
print(os.path.getsize('../data/test_data_20230130-142810.p'))

171216
